# Pydough LLM Demo

This notebook showcases how an LLM can generate PyDough queries from natural language instructions. The goal is to demonstrate how AI can automate complex data analysis, making querying faster, more intuitive, and accessible without needing deep technical expertise.

Each example highlights different capabilities, including aggregations, filtering, ranking, and calculations across multiple collections.

## Setup

First, we import the created client.

In [2]:
from llm import LLMClient

Then we initialize the client.

In [3]:
client = LLMClient()

One should use the `ask()` method to make a query to the model.

We get a `result` object with the next attributes: 

- `code`:  The PyDough query generated by the LLM.
- `full_explanation`: A detailed explanation of how the query works.
- `df`:The dataframe containing the query results.
- `exception`: Stores any errors encountered while executing the query.
- `original_question`: The natural language question input by the user.
- `sql`: The SQL equivalent of the generated PyDough query.
- `base_prompt`: The initial instruction given to the LLM to generate the query.
- `cheat_sheet`:  A reference guide or example queries to help the LLM structure responses.
- `knowledge_graph`: The metadata structure that informs the LLM about available collections and relationships.
 


### Example:

First, we give the client the query we need pydough code for.

In [18]:
result= client.ask("Give me all the suppliers name from United States")

After that, we can consult all the necessary atributes from the result.

At first, I want the pydough code with a **full explanation**.

In [22]:
print(result.full_explanation)

I'll create a PyDough code snippet to retrieve all supplier names from the United States.

First, I need to find suppliers whose nation is the United States, then select just their names.

```python
# Find all suppliers from the United States
us_suppliers = suppliers.WHERE(nation.name == "UNITED STATES").CALCULATE(
    supplier_name=name
)
```

This query:
1. Starts with the suppliers collection
2. Filters to only include suppliers where the associated nation's name is "UNITED STATES"
3. Returns only the supplier name field, renamed as "supplier_name" for clarity


We can also ask for the pydough **code** without the explanation.

In [23]:
print(result.code)

# Find all suppliers from the United States
us_suppliers = suppliers.WHERE(nation.name == "UNITED STATES").CALCULATE(
    supplier_name=name
)


And if we want to visually check, analyze or edit the resulting **dataframe**, we also can.

In [24]:
result.df

,supplier_name
0,Supplier#000000010
1,Supplier#000000019
2,Supplier#000000046
3,Supplier#000000049
4,Supplier#000000055
...,...
388,Supplier#000009819
389,Supplier#000009829
390,Supplier#000009859
391,Supplier#000009906


We can also check the original natural language **question** that was asked.

In [19]:
print(result.original_question)

Give me all the suppliers name from United States


And if we want to compare, we can get the **equivalent SQL query** created by Pydough.

In [21]:
print(result.sql)

SELECT
  name AS supplier_name
FROM (
  SELECT
    _table_alias_0.name AS name,
    _table_alias_1.name AS name_3
  FROM (
    SELECT
      s_name AS name,
      s_nationkey AS nation_key
    FROM main.SUPPLIER
  ) AS _table_alias_0
  LEFT JOIN (
    SELECT
      n_nationkey AS key,
      n_name AS name
    FROM main.NATION
  ) AS _table_alias_1
    ON nation_key = key
)
WHERE
  name_3 = 'UNITED STATES'


If we need to see the **base instruction** that guided the LLM, we can.

In [22]:
print(result.base_prompt)



You are an AI tasked with converting natural language descriptions into PyDough code snippets. You will be provided with two reference files: 


1. **PyDough Reference File** - This file contains the core concepts, functions, and syntax of PyDough.
{script_content}

2. **Database Structure Reference File** - This file outlines the database schema, collections, fields, and relationships.
{database_content}

Your objective is to analyze the provided natural language description that outlines a database query or manipulation task and generate a corresponding PyDough code snippet that adheres to the syntax and structure in the PyDough Reference File.

**Instructions:**
1. Extract the main components of the natural language description to identify the database query or manipulation required.
2. Generate PyDough code that:
   - Uses clear and concise syntax using the correct functions, parameters, and structure.
   - Avoids the bad examples reference in the Pydough Reference File
   - Prop

We also have a reference guide or **cheat_sheet** with example queries to help structure responses.

In [23]:
print(result.cheat_sheet)

# PYDOUGH CHEAT SHEET

**1. COLLECTIONS & SUB-COLLECTIONS**  

- **Syntax**: Access collections/sub-collections using dot notation.  

- **Examples**:  
  - `People` → Access all records in the 'People' collection.  
  - `People.current_address` → Access current addresses linked to people.  
  - `Packages.customer` → Access customers linked to packages.  

- **Warnings**:  
  - Sub-collections must exist in the metadata graph (e.g., `People.packages` is valid; undefined sub-collections like `People.orders` are invalid).  
  - Avoid reassigning collection names to variables (e.g., `Addresses = 42` breaks subsequent access).

**2. CALCULATE EXPRESSIONS**  

- **Purpose**: Derive new fields, rename existing ones or select specific fields.  
  The value of one of these terms in a `CALCULATE` must be expressions that are singular with regards to the current context. That can mean:
  - Referencing one of the scalar properties of the current collection.
  - Creating a literal.
  - Referencing

And if we want to understand the collections and relationships used, we can check the Pydough **metadata**.

In [24]:
print(result.knowledge_graph)

### The `customers` collection contains the following columns:
- **acctbal**: The account balance of the customer.
- **address**: The address of the customer.
- **comment**: Additional comments or notes about the customer.
- **key**: A unique identifier for the customer.
- **mktsegment**: The market segment the customer belongs to.
- **name**: The name of the customer.
- **nation_key**: A foreign key referencing the `nations` collection.
- **phone**: The customer's phone number.
- **nation**: The corresponding nation of the customer (reverse of `nations.customers`).
- **orders**: A list of all orders placed by the customer (reverse of `orders.customer`).

### The `lines` collection contains the following columns:
- **comment**: Additional comments or notes about the line item.
- **commit_date**: The committed delivery date for the line item.
- **discount**: The discount applied to the line item.
- **extended_price**: The extended price of the line item.
- **line_number**: The line numb

The next example is one with an **exception**.

In [25]:
result= client.ask("For each of the 5 largest part sizes, find the part of that size with the largest retail price")

print(result.full_explanation)

I'll analyze this request and create a PyDough code snippet to find the part with the largest retail price for each of the 5 largest part sizes.

First, I need to:
1. Group parts by their size
2. For each size, find the part with the highest retail price
3. Sort by part size in descending order
4. Take only the top 5 sizes

Here's the PyDough code to accomplish this:

```python
# First, calculate the maximum retail price for each part size
max_price_by_size = GROUP_BY(parts.CALCULATE(size=size, retail_price=retail_price), 
                            name="parts_group", 
                            by=size).CALCULATE(
    size=size,
    max_retail_price=MAX(parts_group.retail_price)
)

# Join back to the parts collection to get the part details with the maximum retail price
parts_with_max_price = parts.CALCULATE(
    size=size,
    name=name,
    retail_price=retail_price,
    part_key=key
).WHERE(
    (size == max_price_by_size.size) & 
    (retail_price == max_price_by_size.max_retai

If one calls the dataframe and gets an error, no response, or a empty dataframe, it is possible that there is a PyDough exception. We can check this by running:

In [26]:
print(result.exception)

An error occurred while processing the code: Unrecognized term of simple table collection 'parts' in graph 'TPCH': 'parts'


You can try to fix the error using the `correct` method. We are going to declare a new variable to obtain the corrected result. 

In [27]:
corrected_result = client.correct(result)

To see how the model try to solve the issue, you can print the full explanation of the `corrected_result`.

In [28]:
print(corrected_result.full_explanation)

I see the issue with the code. The error message indicates that the collection 'parts' is not recognized in the graph 'TPCH'. Looking at the database structure reference file, I can see that the correct collection name is indeed available, but there's an issue with how we're accessing it.

Let me fix the code:

```python
# First, calculate the maximum retail price for each part size
max_price_by_size = GROUP_BY(parts.CALCULATE(size=size, retail_price=retail_price), 
                            name="parts_group", 
                            by=size).CALCULATE(
    size=size,
    max_retail_price=MAX(parts_group.retail_price)
)

# Join back to the parts collection to get the part details with the maximum retail price
parts_with_max_price = parts.CALCULATE(
    size=size,
    name=name,
    retail_price=retail_price,
    part_key=key
).WHERE(
    retail_price == max_price_by_size.WHERE(size == parts.size).max_retail_price
)

# Get the top 5 largest part sizes with their highest retail p

Note: You can try this as many times as you like if an exception keeps ocurring. 

## Test Cases

### Customer Segmentation.

#### 1. Find the names of all customers and the number of orders placed in 1995 in Europe.

Demonstrates simple filtering, counting, and sorting while being business-relevant for regional market analysis. Adds a second filtering layer by including account balance and order activity, making it more dynamic.

In [29]:
query= "Find the names of all customers and the number of orders placed in 1995 in Europe."

result= client.ask(query)

print(result.full_explanation)
result.df.head()

I'll create a PyDough code snippet to find the names of all customers and the number of orders they placed in 1995 in Europe.

```python
# Find customers in Europe
# Count orders placed in 1995 for each customer
european_customers_1995_orders = customers.WHERE(
    nation.region.name == "EUROPE"
).CALCULATE(
    customer_name=name,
    order_count=COUNT(orders.WHERE(YEAR(order_date) == 1995))
)
```

This code:
1. Starts with the `customers` collection
2. Filters for customers in Europe using the `WHERE` clause
3. Uses `CALCULATE` to:
   - Include the customer name
   - Count the number of orders placed in 1995 using a nested `WHERE` clause with the `YEAR` function

The result will contain two columns: `customer_name` and `order_count` for all European customers, showing how many orders each placed in 1995.


,customer_name,order_count
0,Customer#000000011,1
1,Customer#000000015,0
2,Customer#000000018,0
3,Customer#000000020,3
4,Customer#000000026,2


**Follow up**: Now, give me the ones who have an account balance greater than $700 and placed at least one order in that same year. Sorted in descending order by the number of orders.

In [30]:
result= client.discourse(result, 
"""Now, give me the ones who have an account balance greater than $700 and placed at least one order in that same year. 
Sorted in descending order by the number of orders.""")

print(result.full_explanation)
result.df.head()

I'll create a PyDough code snippet to find European customers with account balances greater than $700 who placed at least one order in 1995, sorted by their order count in descending order.

```python
# First, identify orders placed in 1995
orders_1995 = orders.WHERE(YEAR(order_date) == 1995)

# Find European customers with account balance > $700 who placed at least one order in 1995
european_customers = customers.CALCULATE(
    customer_name=name,
    account_balance=acctbal,
    num_orders_1995=COUNT(orders.WHERE(YEAR(order_date) == 1995))
).WHERE(
    (nation.region.name == "EUROPE") &
    (acctbal > 700) &
    (num_orders_1995 > 0)
).ORDER_BY(
    num_orders_1995.DESC()
)
```

This code:
1. Creates a contextless expression for orders placed in 1995
2. Calculates each customer's name, account balance, and number of orders placed in 1995
3. Filters for customers who:
   - Are located in Europe (via the nation-region relationship)
   - Have an account balance greater than $700
   - Pl

,customer_name,account_balance,num_orders_1995
0,Customer#000107440,2464.61,12
1,Customer#000014920,7026.03,11
2,Customer#000079606,989.51,11
3,Customer#000108496,7388.38,11
4,Customer#000009019,2247.06,10


### 2. List customers who ordered in 1996 but not in 1997, with a total spent of over 1000$?

Showcases PyDough’s HAS() and HASNOT() functions, helping analyze customer retention and spending patterns. Also incorporates a time-based calculation.

In [10]:
query= "List customers who ordered in 1996 but not in 1997 with a total spent of over 1000$ If you use HAS add a ==1 for it to work"

result= client.ask(query)

print(result.full_explanation)
result.df.head()

I'll create a PyDough code snippet to find customers who ordered in 1996 but not in 1997, with a total spent of over $1000.

```python
# First, identify customers who ordered in 1996
customers_1996 = customers.WHERE(
    HAS(orders.WHERE(YEAR(order_date) == 1996)) == 1
)

# Calculate total spent in 1996 for these customers
customers_with_spend = customers_1996.CALCULATE(
    customer_key=key,
    customer_name=name,
    total_spent_1996=SUM(orders.WHERE(YEAR(order_date) == 1996).total_price),
    ordered_in_1997=HAS(orders.WHERE(YEAR(order_date) == 1997)) == 1
)

# Filter for customers who didn't order in 1997 and spent over $1000 in 1996
result = customers_with_spend.WHERE(
    (ordered_in_1997 == False) & 
    (total_spent_1996 > 1000)
).CALCULATE(
    customer_key,
    customer_name,
    total_spent_1996
)
```

This code:
1. First identifies customers who placed orders in 1996
2. Calculates their total spending in 1996 and checks if they ordered in 1997
3. Filters for customers who 

,customer_key,customer_name,total_spent_1996
0,5,Customer#000000005,98790.79
1,17,Customer#000000017,481076.02
2,20,Customer#000000020,538764.56
3,31,Customer#000000031,720459.29
4,38,Customer#000000038,788657.01


**Follow up**: Include the number of months since the last order and sort by total spent, highest first.

In [9]:
result= client.discourse(result,
"Include the number of months since the last order and sort by total spent, highest first.")

print(result.full_explanation)
result.df.head()

I'll create a PyDough code snippet to find customers who ordered in 1996 but not in 1997, with total spending over $1000, including the number of months since their last order, sorted by total spent in descending order.

```python
# First, identify customers who ordered in 1996
customers_1996 = customers.WHERE(
    HAS(orders.WHERE(YEAR(order_date) == 1996)) == 1
)

# Then filter to exclude those who ordered in 1997
target_customers = customers_1996.WHERE(
    HAS(orders.WHERE(YEAR(order_date) == 1997)) == 0
)

# Calculate total spent and months since last order
result = target_customers.CALCULATE(
    customer_name=name,
    total_spent=SUM(orders.total_price),
    last_order_date=MAX(orders.order_date),
    months_since_last_order=DATEDIFF("months", MAX(orders.order_date), DATETIME("now"))
).WHERE(
    total_spent > 1000
).ORDER_BY(
    total_spent.DESC()
)
```

This code:
1. First identifies customers who placed orders in 1996
2. Then filters out those who also placed orders in 1997

,customer_name,total_spent,last_order_date,months_since_last_order
0,Customer#000001948,5614411.17,1998-06-26,321
1,Customer#000047401,5262679.14,1998-04-03,323
2,Customer#000094354,5234152.55,1998-07-08,320
3,Customer#000120877,5200194.28,1998-07-17,320
4,Customer#000012595,4973507.88,1998-07-30,320


### Sales Performance

#### 3. Find the region name with the highest total order value in 1996.

The total order value is defined as potential revenue, defined as the sum of extended_price * (1 - discount)

It introduces precise calculations within the query, ensuring revenue insights.

In [11]:
query="""Find the region name with the highest total order value in 1996. 
The total order value is defined as potential revenue, defined as the sum of extended_price * (1 - discount)"""

result= client.ask(query)

print(result.full_explanation)
result.df.head()

I'll create a PyDough code snippet to find the region name with the highest total order value in 1996, where total order value is defined as the sum of extended_price * (1 - discount).

First, I need to:
1. Filter orders from 1996
2. Calculate the potential revenue for each line item
3. Sum these values by region
4. Find the region with the highest total

```python
# First, calculate the potential revenue for each line item in orders from 1996
orders_1996 = orders.WHERE(YEAR(order_date) == 1996)
lines_1996 = orders_1996.lines.CALCULATE(
    potential_revenue=extended_price * (1 - discount)
)

# Group by region and calculate total potential revenue
region_revenue = regions.CALCULATE(
    region_name=name,
    total_revenue=SUM(nations.customers.orders.WHERE(YEAR(order_date) == 1996).lines.extended_price * (1 - nations.customers.orders.WHERE(YEAR(order_date) == 1996).lines.discount))
)

# Find the region with the highest total revenue
highest_revenue_region = region_revenue.TOP_K(1, by=t

,region_name,total_revenue
0,EUROPE,6.746079e+09


### Product Trends

#### 4. Which 10 customers purchased the highest quantity of products during 1998?

Highlights ranking queries (TOP_K()), customer segmentation, and purchasing trends. 

In [12]:
query= "Which 10 customers purchased the highest quantity of products during 1998?"

result= client.ask(query)

print(result.full_explanation)
result.df.head()

I'll create a PyDough code snippet to find the 10 customers who purchased the highest quantity of products during 1998.

```python
# Filter orders to only include those from 1998
orders_1998 = orders.WHERE(YEAR(order_date) == 1998)

# Calculate the total quantity purchased by each customer in 1998
customer_quantities = customers.CALCULATE(
    customer_key=key,
    customer_name=name,
    total_quantity=SUM(orders_1998.lines.quantity)
)

# Get the top 10 customers by total quantity purchased
top_10_customers = customer_quantities.TOP_K(10, by=total_quantity.DESC())
```

This code:
1. First filters orders to only include those from 1998
2. Then calculates the total quantity of products purchased by each customer from those 1998 orders
3. Finally, selects the top 10 customers based on their total purchased quantity in descending order


,customer_key,customer_name,total_quantity
0,119539,Customer#000119539,1206
1,26518,Customer#000026518,1158
2,24877,Customer#000024877,1104
3,19,Customer#000000019,1084
4,106000,Customer#000106000,1058


**Follow up**: Now only the ones that have "green" on the product name.

In [14]:
result= client.discourse(result, "Only the ones that have 'green' on the product name.")

print(result.full_explanation)
result.df.head()

I'll create a PyDough code snippet to find the top 10 customers who purchased the highest quantity of products during 1998, but only considering products that have 'green' in their name.

```python
# First, filter orders from 1998 with green products and calculate total quantity per customer
customer_green_quantities = customers.CALCULATE(
    customer_key=key,
    customer_name=name,
    total_green_quantity=SUM(
        orders.WHERE(YEAR(order_date) == 1998)
        .lines.WHERE(CONTAINS(part.name, "green"))
        .quantity
    )
).WHERE(total_green_quantity > 0)

# Get the top 10 customers by green product quantity
top_10_green_customers = customer_green_quantities.TOP_K(
    10, 
    by=total_green_quantity.DESC()
).CALCULATE(
    customer_key,
    customer_name,
    total_green_quantity
)
```

This code:
1. Starts with the customers collection
2. Calculates for each customer:
   - Their key and name
   - The total quantity of products they purchased that have 'green' in the name

,customer_key,customer_name,total_green_quantity
0,11509,Customer#000011509,208
1,22147,Customer#000022147,201
2,27577,Customer#000027577,201
3,109117,Customer#000109117,176
4,16252,Customer#000016252,172


### Revenue Performance

#### 5. What is the february 1996 SPM for the almond antique blue royal burnished part in China?

SPM (Selling Profit Margin) = (Total Amount from Sells - (Tax + Commission)) / Total Amount from Sells * 100

This query was provided as a representative example of potential stakeholder inquiries.

Showcases advanced partitioning and filtering, demonstrating how PyDough can be used for highly specific business KPIs. Compare with a previous time period and exclude/include specific suppliers, making it a progressive data exploration example.

In [15]:
query= """What is the february 1996 SPM for the almond antique blue royal burnished part in China? 
SPM (Selling Profit Margin) = (Total Amount from Sells - (Tax + Commission)) / Total Amount from Sells * 100"""

result= client.ask(query)

print(result.full_explanation)
result.df.head()

I need to create a PyDough query to calculate the February 1996 Selling Profit Margin (SPM) for the "almond antique blue royal burnished" part in China.

First, I'll need to:
1. Find the specific part by its name
2. Filter for line items from February 1996
3. Filter for suppliers from China
4. Calculate the SPM using the formula provided

Here's the PyDough code:

```python
# Find the part with the specified name
target_part = parts.WHERE(
    name == "almond antique blue royal burnished"
)

# Calculate the SPM for February 1996 in China
spm_result = target_part.lines.WHERE(
    (YEAR(ship_date) == 1996) & 
    (MONTH(ship_date) == 2) & 
    (supplier.nation.name == "CHINA")
).CALCULATE(
    part_name=target_part.name,
    total_sales=SUM(extended_price),
    total_tax=SUM(tax),
    total_commission=SUM(discount * extended_price),  # Using discount as commission
    spm=((SUM(extended_price) - (SUM(tax) + SUM(discount * extended_price))) / 
         SUM(extended_price)) * 100
)
```

Th

AttributeError: 'NoneType' object has no attribute 'head'

**Follow up**: Compare that to november 1995 SPM, have we seen an increase?

In [16]:
query+= """Compare that to november 1995 SPM, have we seen an increase?"""

result= client.ask(query)

print(result.full_explanation)
result.df.head()

I'll analyze this request and create PyDough code to calculate the Selling Profit Margin (SPM) for the specified part in China for February 1996 and November 1995.

First, I need to identify the "almond antique blue royal burnished" part, then find sales in China for the specified months, and calculate the SPM according to the formula:

SPM = (Total Amount from Sells - (Tax + Commission)) / Total Amount from Sells * 100

Note: I don't see a "commission" field in the database structure. I'll assume the commission is represented by the discount field, as this is typically subtracted from the selling price.

```python
# First, find the specific part by its name
target_part = parts.WHERE(
    name == "almond antique blue royal burnished"
).CALCULATE(key, name)

# Calculate SPM for February 1996 sales in China
feb_1996_spm = lines.WHERE(
    (YEAR(ship_date) == 1996) & 
    (MONTH(ship_date) == 2) & 
    (part.key == target_part.key) &
    (supplier.nation.name == "CHINA")
).CALCULATE(
    

AttributeError: 'NoneType' object has no attribute 'head'

**Follow up**: Now exclude supplier Supplier#000001305 and focus only on supplier Supplier#000008802

In [17]:
query+= """Now exclude supplier Supplier#000001305 and focus only on supplier Supplier#000008802"""

result= client.ask(query)

print(result.full_explanation)
result.df.head()

I'll analyze this request and generate PyDough code to calculate the Selling Profit Margin (SPM) for the specified part and suppliers.

First, I need to:
1. Find the almond antique blue royal burnished part
2. Calculate SPM for February 1996 and November 1995 in China
3. Compare the two periods
4. Filter for specific suppliers as requested

```python
# First, identify the almond antique blue royal burnished part
target_part = parts.WHERE(
    CONTAINS(name, "almond") & 
    CONTAINS(name, "antique") & 
    CONTAINS(name, "blue") & 
    CONTAINS(name, "royal") & 
    CONTAINS(name, "burnished")
)

# Calculate SPM for February 1996
feb_1996_spm = lines.WHERE(
    (part_key == target_part.key) &
    (YEAR(ship_date) == 1996) & 
    (MONTH(ship_date) == 2) &
    (supplier.nation.name == "CHINA")
).CALCULATE(
    total_sales=SUM(extended_price),
    total_tax=SUM(tax),
    total_commission=SUM(extended_price * discount),  # Using discount as commission
    spm=(SUM(extended_price) - (SUM(ta

AttributeError: 'NoneType' object has no attribute 'head'